# **ASSIGNMENT 04 - OPTIMIZATION**

## **The stigler diet program**

The objective of the optimization programm is to determine how much money needs to be spend of each food at a minimum to satisfy the minimum nutrition requirement. The data consists on the list of foods or comodities along with their cost per unit and the nutrient content. 

--->>[Commodity, Unit, 1939 price (cents), Calories (kcal), Protein (g),
Calcium (g), Iron (mg), Vitamin A (KIU), Vitamin B1 (mg), Vitamin B2 (mg),
Niacin (mg), Vitamin C (mg)]

In the dateset we are also provided with the minimum daily requirements for 8 of the nutrients needed. --->> [Nutrient minimums]

This program solves the Stigler Diet Program
The problem was studied by the first time by the Nobel laureate Stigler as a family of problem
He didn't mean to recommed a diet but to illustrate an economics general problem.

The idea is that there exist a minimum quantities of nutrients and calories that have to be met by everyday ingestion. These quantities are reflected in the table 'nutrients' in the data.py file.
There are known quantities of all these nutrients in a set of foods as reflected in the table 'data' in the file data.py.

In [32]:
# We will use ortools to solve this problem
# ortools have been developed by Google to solve many kinds of mathematical programming (optimization) problems
from ortools.linear_solver import pywraplp
from ortools.init import pywrapinit

# We import the file data.py to have access to the definitions in this file
from data import * 

# **EXERCISE 1: Understand and execute the program**

### Our initial goal is to minimize the sum of the foods...

First we crete the linear solver with the GLOP backend. 

In [33]:
solver = pywraplp.Solver.CreateSolver('GLOP')
if not solver:
    exit

In [34]:
# Variables are defined using solver.Numvar. You can create individual variables like
# in the commented sample below
# Create the variables x and y.
x = solver.NumVar(0, solver.infinity(), 'x')
y = solver.NumVar(0, solver.infinity(), 'y')
# The parameters of NumVar are:
#   - minimum value
#   - maximum value
#   - name of variable

# Variables

To minimize the price we will need to declare the variables of the function, these will be the food items (commodity column of the table -->> item[0] holds the name of each food in the 'data' table)

In [35]:
# Because this function is general we could create the variables in an array, to hold
# a large number of variables, declaring an array to hold our variables. Note that
# item[0] holds the name of each food in the 'data' table. The variables itself will hold
# the quantity of each item
foods = []
foods = [solver.NumVar(0.0, solver.infinity(), item[0]) for item in data]
print(foods)

print("----------------------------------------------------------------------")
print('Number of variables =', solver.NumVariables())

[Wheat Flour (Enriched), Macaroni, Wheat Cereal (Enriched), Corn Flakes, Corn Meal, Hominy Grits, Rice, Rolled Oats, White Bread (Enriched), Whole Wheat Bread, Rye Bread, Pound Cake, Soda Crackers, Milk, Evaporated Milk (can), Butter, Oleomargarine, Eggs, Cheese (Cheddar), Cream, Peanut Butter, Mayonnaise, Crisco, Lard, Sirloin Steak, Round Steak, Rib Roast, Chuck Roast, Plate, Liver (Beef), Leg of Lamb, Lamb Chops (Rib), Pork Chops, Pork Loin Roast, Bacon, Ham, smoked, Salt Pork, Roasting Chicken, Veal Cutlets, Salmon, Pink (can), Apples, Bananas, Lemons, Oranges, Green Beans, Cabbage, Carrots, Celery, Lettuce, Onions, Potatoes, Spinach, Sweet Potatoes, Peaches (can), Pears (can), Pineapple (can), Asparagus (can), Green Beans (can), Pork and Beans (can), Corn (can), Peas (can), Tomatoes (can), Tomato Soup (can), Peaches, Dried, Prunes, Dried, Raisins, Dried, Peas, Dried, Lima Beans, Dried, Navy Beans, Dried, Coffee, Tea, Cocoa, Chocolate, Sugar, Corn Syrup, Molasses, Strawberry Preser

# Constraints

In [36]:
# The constraints are reprsented as a matrix where each variable holds
# a coefficient in the linear combination of all constraints. This linear combination
# must fullfill the constraint of being between two values. 
# Let's have a look at one example 0 <= 2*x + y <= 5.
ct = solver.Constraint(0, 5, 'ct')
ct.SetCoefficient(x, 2)
ct.SetCoefficient(y, 1)

Now we need to set the constraints that are proposed by the problem, these are the requiremens that the total amount of nutrients provided by all the foods be at least the minimum daily nutrient requirement for each nutrient. Now the constraints consider the amount of each nutrient provided by each food. We have to notice that is is [i+3] because the nutrient data we have to consider for our constraints start at the 4th column of the list. 

In [37]:
# The constrainsts can be reprsented as arrays to make evrything easier
# Create the constraints, one per nutrient.
constraints = []
for i, nutrient in enumerate(nutrientes):
    # Append one constrint per nutrient. If you look at the nutrient table,
    # you will see that the minimum value is stored at nurient[1] (second column)
    constraints.append(solver.Constraint(nutrient[1], solver.infinity()))

    # The coefficients in this constraint are given by the 'data' table
    # The variable for which we are creating the coefficient is foods[j]
    # The coefficient itself has the value of the corresponding nutrient in 'data'
    # table, which is located at position i + 3 (where i is the nutrient number)
    for j, item in enumerate(data):
        constraints[i].SetCoefficient(foods[j], item[i + 3])

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 10


# Objective + invoking the solver 

In [38]:
# Finally, we set the objective function
# As usual, let's see in comments a simple objective function for two variables
# Again, we specify the coefficients of each variable for the linear combination of all variables
# Create the objective function, 3 * x + y.
objective = solver.Objective()
objective.SetCoefficient(x, 3)
objective.SetCoefficient(y, 1)
objective.SetMaximization()

The last step is to create the objective function of the optimisation problem. The objective function will be the total cost of the food, which accounts for the sum of all variables. 

In [39]:
# Let's translate this to our problem
# Objective function: Minimize the sum of foods cost. 
# The objective function is the sum of each variable (food quantity) multiplied 
# by its cost (cost is in column 3). The costs are the coefficients
objective = solver.Objective()
for f, food in enumerate(foods):
    objective.SetCoefficient(food, 1) # 2 is column 3
objective.SetMinimization()

status = solver.Solve()

In [40]:
# Check that the problem has an optimal solution.
if status != solver.OPTIMAL:
    print('The problem does not have an optimal solution!')
    if status == solver.FEASIBLE:
        print('A potentially suboptimal solution was found.')
    else:
        print('The solver could not solve the problem.')
        exit(1)
else:
    print('Optimal solution found')

Optimal solution found


In [41]:
# Display the amounts to purchase of each food.
nutrients_result = [0] * len(nutrientes)
print('\nDaily Foods:')
for i, food in enumerate(foods):
    if food.solution_value() > 0.0:
        print('{}: {}'.format(data[i][0], food.solution_value()))
        for j, _ in enumerate(nutrientes):
            nutrients_result[j] += data[i][j + 3] * food.solution_value()

print('\nOptimal cost: ${:.4f}'.format(objective.Value()))

print('\nNutrients per day:')
for i, nutrient in enumerate(nutrientes):
    print('{}: {:.2f} (min {})'.format(nutrient[0], nutrients_result[i], nutrient[1]))


Daily Foods:
Wheat Flour (Enriched): 44.608754452836955
Spinach: 5.444250871080141

Optimal cost: $50.0530

Nutrients per day:
Calories (kcal): 2000.00 (min 2000)
Protein (g): 63520.04 (min 70)
Calcium (g): 89.22 (min 0.8)
Iron (mg): 17033.50 (min 12)
Vitamin A (KIU): 5000.00 (min 5000)
Vitamin B1 (mg): 2502.36 (min 1.8)
Vitamin B2 (mg): 1560.60 (min 2.7)
Niacin (mg): 19852.12 (min 18)
Vitamin C (mg): 14998.91 (min 75)


Here we can see: 
- The list of foods that would have to be purchased for the optimum solution and the cost of each food. 
- The Optimal cost (50.0530$) which is the value of the objective function, solution to the problem
- The total amount of nutrients gathered for the solution of the problem

# **EXERCISE 2:Modify the program the data to consider not only minimum nutrients quantities but maximum as well.** 

## **OPTION 1:** For this option, what I have done is just changing the solvers objective from minimisation to maximisation -> we will get a not feasible solution, the solver won't be able to find the most optimum solution

First we import the data and create the linear solver with the GLOP backend.

In [42]:
# We import the file data.py to have access to the definitions in this file
from data import * 

# Create the linear solver with the GLOP backend.
# GLOP is an open source solver which can be used freely
solver = pywraplp.Solver.CreateSolver('GLOP')
if not solver:
    exit

In [43]:
foods = [solver.NumVar(0.0, solver.infinity(), item[0]) for item in data]
print(foods)


print("----------------------------------------------------------------------")
print('Number of variables =', solver.NumVariables())


[Wheat Flour (Enriched), Macaroni, Wheat Cereal (Enriched), Corn Flakes, Corn Meal, Hominy Grits, Rice, Rolled Oats, White Bread (Enriched), Whole Wheat Bread, Rye Bread, Pound Cake, Soda Crackers, Milk, Evaporated Milk (can), Butter, Oleomargarine, Eggs, Cheese (Cheddar), Cream, Peanut Butter, Mayonnaise, Crisco, Lard, Sirloin Steak, Round Steak, Rib Roast, Chuck Roast, Plate, Liver (Beef), Leg of Lamb, Lamb Chops (Rib), Pork Chops, Pork Loin Roast, Bacon, Ham, smoked, Salt Pork, Roasting Chicken, Veal Cutlets, Salmon, Pink (can), Apples, Bananas, Lemons, Oranges, Green Beans, Cabbage, Carrots, Celery, Lettuce, Onions, Potatoes, Spinach, Sweet Potatoes, Peaches (can), Pears (can), Pineapple (can), Asparagus (can), Green Beans (can), Pork and Beans (can), Corn (can), Peas (can), Tomatoes (can), Tomato Soup (can), Peaches, Dried, Prunes, Dried, Raisins, Dried, Peas, Dried, Lima Beans, Dried, Navy Beans, Dried, Coffee, Tea, Cocoa, Chocolate, Sugar, Corn Syrup, Molasses, Strawberry Preser

Creating the constraints

In [44]:
# The constrainsts can be reprsented as arrays to make evrything easier
# Create the constraints, one per nutrient.
constraints = []
for i, nutrient in enumerate(nutrientes):
    # Append one constrint per nutrient. If you look at the nutrient table,
    # you will see that the minimum value is stored at nurient[1] (second column)
    constraints.append(solver.Constraint(nutrient[1], solver.infinity()))

    # The coefficients in this constraint are given by the 'data' table
    # The variable for which we are creating the coefficient is foods[j]
    # The coefficient itself has the value of the corresponding nutrient in 'data'
    # table, which is located at position i + 3 (where i is the nutrient number)
    for j, item in enumerate(data):
        constraints[i].SetCoefficient(foods[j], item[i + 3])

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 9


The last step is to create the objective function of the optimisation problem. The objective function will be the total cost of the food, which accounts for the sum of all variables. 
### In the last exercise, the objective function was set to minimisation, now we set it for maximisation

In [45]:
# Let's translate this to our problem
# Objective function: Minimize the sum of foods cost. 
# The objective function is the sum of each variable (food quantity) multiplied 
# by its cost (cost is in column 3). The costs are the coefficients
objective = solver.Objective()
for f, food in enumerate(foods):
    objective.SetCoefficient(food, 1) # 2 is column 3
objective.SetMaximization()

status = solver.Solve()

In [46]:
# Check that the problem has an optimal solution.
if status != solver.OPTIMAL:
    print('The problem does not have an optimal solution!')
    if status == solver.FEASIBLE:
        print('A potentially suboptimal solution was found.')
    else:
        print('The solver could not solve the problem.')
        exit(1)
else:
    print('Optimal solution found')

The problem does not have an optimal solution!
The solver could not solve the problem.


### As we expected for this part of the problem, the problem does not have a feasible solution

## **OPTION 2:** adding a new value for each of the nutrients, this value with be the upper bound of the constraints. As a consequence, we will create constraints that are type <= >=. 

In [47]:
# We import the file data.py to have access to the definitions in this file
from data import * 

# Create the linear solver with the GLOP backend.
# GLOP is an open source solver which can be used freely
solver = pywraplp.Solver.CreateSolver('GLOP')
if not solver:
    exit

In [48]:
foods = [solver.NumVar(0.0, solver.infinity(), item[0]) for item in data]
print(foods)


print("----------------------------------------------------------------------")
print('Number of variables =', solver.NumVariables())

[Wheat Flour (Enriched), Macaroni, Wheat Cereal (Enriched), Corn Flakes, Corn Meal, Hominy Grits, Rice, Rolled Oats, White Bread (Enriched), Whole Wheat Bread, Rye Bread, Pound Cake, Soda Crackers, Milk, Evaporated Milk (can), Butter, Oleomargarine, Eggs, Cheese (Cheddar), Cream, Peanut Butter, Mayonnaise, Crisco, Lard, Sirloin Steak, Round Steak, Rib Roast, Chuck Roast, Plate, Liver (Beef), Leg of Lamb, Lamb Chops (Rib), Pork Chops, Pork Loin Roast, Bacon, Ham, smoked, Salt Pork, Roasting Chicken, Veal Cutlets, Salmon, Pink (can), Apples, Bananas, Lemons, Oranges, Green Beans, Cabbage, Carrots, Celery, Lettuce, Onions, Potatoes, Spinach, Sweet Potatoes, Peaches (can), Pears (can), Pineapple (can), Asparagus (can), Green Beans (can), Pork and Beans (can), Corn (can), Peas (can), Tomatoes (can), Tomato Soup (can), Peaches, Dried, Prunes, Dried, Raisins, Dried, Peas, Dried, Lima Beans, Dried, Navy Beans, Dried, Coffee, Tea, Cocoa, Chocolate, Sugar, Corn Syrup, Molasses, Strawberry Preser

Now, to set the constraints, notice how I plug in the new value I have added for the data set: **nutrients[2]**

This is what I have changed in the dataset: 

nutrients = [

    ['Calories (kcal)', 2000, 5000],

    ['Protein (g)', 70, 170],

    ['Calcium (g)', 0.8, 2.7],

    ['Iron (mg)', 12, 26],

    ['Vitamin A (KIU)', 5000, 9000],

    ['Vitamin B1 (mg)', 1.8, 3.4],

    ['Vitamin B2 (mg)', 2.7, 5.8],

    ['Niacin (mg)', 18, 32],

    ['Vitamin C (mg)', 75, 210],

In [49]:
# The constrainsts can be reprsented as arrays to make evrything easier
# Create the constraints, one per nutrient.
nutrients = [
    ['Calories (kcal)', 2000, 5000],
    ['Protein (g)', 70, 170],
    ['Calcium (g)', 0.8, 2.7],
    ['Iron (mg)', 12, 26],
    ['Vitamin A (KIU)', 5000, 9000],
    ['Vitamin B1 (mg)', 1.8, 3.4],
    ['Vitamin B2 (mg)', 2.7, 5.8],
    ['Niacin (mg)', 18, 32],
    ['Vitamin C (mg)', 75, 210],
]
constraints = []
for i, nutrient in enumerate(nutrients):
    # Append one constrint per nutrient. If you look at the nutrient table,
    # you will see that the minimum value is stored at nurient[1] and maximum value at nutrient[2] (second and third column)
    constraints.append(solver.Constraint(nutrient[1], nutrient[2], 'ct'))

    # The coefficients in this constraint are given by the 'data' table
    # The variable for which we are creating the coefficient is foods[j]
    # The coefficient itself has the value of the corresponding nutrient in 'data'
    # table, which is located at position i + 3 (where i is the nutrient number)
    for j, item in enumerate(data):
        constraints[i].SetCoefficient(foods[j], item[i + 3])

print('Number of constraints =', solver.NumConstraints())

Number of constraints = 9


In [50]:
# Let's translate this to our problem
# Objective function: Minimize the sum of foods cost. 
# The objective function is the sum of each variable (food quantity) multiplied 
# by its cost (cost is in column 3). The costs are the coefficients
objective = solver.Objective()
for f, food in enumerate(foods):
    objective.SetCoefficient(food, 1) # 2 is column 3
objective.SetMinimization()

status = solver.Solve()

In [51]:
# Check that the problem has an optimal solution.
if status != solver.OPTIMAL:
    print('The problem does not have an optimal solution!')
    if status == solver.FEASIBLE:
        print('A potentially suboptimal solution was found.')
    else:
        print('The solver could not solve the problem.')
        exit(1)
else:
    print('Optimal solution found')

The problem does not have an optimal solution!
The solver could not solve the problem.


### As we can see, setting both a maximum and minimum threshold for the nutrients the answer does not give us a feasible anwer. The reason is, as we could see in exercise 1, for example, the total amount of proteins that the foods provided was 63520.04 and now I have set a maximum value for the proteins to be 170, much lower than that, this is one of the constraints that doesn't allow the solver to find a feassible solution, because it is not possible.